In [1]:
import pandas as pd

In [2]:
full_df = pd.read_csv('dataset/train.csv')
df = full_df.copy()

In [3]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

def update_train_df(dataframe):
    imputers = {
        'name_imputer': SimpleImputer(strategy='constant', fill_value='Unknown Unknown'),
        'num_imputer': SimpleImputer(strategy='median'),
        'cat_imputer': SimpleImputer(strategy='constant', fill_value='Unknown'),
    }

    num_col = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    cat_cols = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']
    
    df = dataframe.copy()

    # Impute Part
    df.loc[:, 'Name'] = imputers['name_imputer'].fit_transform(df.loc[:, 'Name'].to_frame())
    df.loc[:, num_col] = imputers['num_imputer'].fit_transform(df.loc[:, num_col])
    df.loc[:, cat_cols] = imputers['cat_imputer'].fit_transform(df.loc[:, cat_cols])

    # Feature engineering Part
    df.loc[:, 'FirstName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[0])
    df.loc[:, 'LastName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[1])

    df.loc[:, 'PassengerGGGG'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[0]))
    df.loc[:, 'PassengerPP'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[1]))
    
    df.loc[:, 'NumberOfFellows'] = df.groupby('PassengerGGGG')['PassengerGGGG'].transform('count')
    df.loc[:, 'IsAlone'] = df.loc[:, 'NumberOfFellows'].apply(lambda x: 1 if x == 1 else 0)

    df['cabin_deck'] = df['Cabin'].apply(lambda x: x.split('/')[0] if x != 'Unknown' else 'U')
    df['cabin_num'] = df['Cabin'].apply(lambda x: int(x.split('/')[1]) if x != 'Unknown' else -1)
    df['cabin_side'] = df['Cabin'].apply(lambda x: x.split('/')[2] if x != 'Unknown' else 'U')

    df.loc[:, 'TotalSpend'] = df.loc[:, ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    df['IsSpendingZero'] = (df['TotalSpend'] == 0).astype(int)

    # Removing unnecessary columns
    df.drop(['Cabin', 'Name', 'PassengerId'], inplace=True, axis=1)
    df.drop(['FirstName', 'LastName', 'PassengerGGGG'], inplace=True, axis=1)


    # std_scalers = {}

    # num_cols = df.select_dtypes('float').columns

    # for num_col in num_cols:
    #     scaler = StandardScaler()
    #     df.loc[:, num_col] = scaler.fit_transform(df.loc[:, num_col])

    #     std_scalers = 

    # Encoding 
    cat_data = df.select_dtypes('object')
    cat_data.loc[:, ['CryoSleep', 'VIP']] = cat_data.loc[:, ['CryoSleep', 'VIP']].astype('str')
    
    one_hot_encoder = OneHotEncoder(drop='first', dtype=int)
    
    cat_data_encoded = one_hot_encoder.fit_transform(cat_data)
    encoded_feature_names = one_hot_encoder.get_feature_names_out(cat_data.columns)
    
    cat_encoded = pd.DataFrame(cat_data_encoded.toarray(), columns = encoded_feature_names, index=cat_data.index)
    
    non_cat = df.drop(cat_data.columns, axis=1)
    df = pd.concat([non_cat, cat_encoded], axis=1)
    
    df.loc[:, 'Transported'] = df.loc[:, 'Transported'].astype(int)
    
    return df, imputers, one_hot_encoder

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

def update_test_df(dataframe, imputers, one_hot_encoder):
    num_col = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    cat_cols = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']
    
    df = dataframe.copy()

    # Impute Part
    df.loc[:, 'Name'] = imputers['name_imputer'].transform(df.loc[:, 'Name'].to_frame())
    df.loc[:, num_col] = imputers['num_imputer'].transform(df.loc[:, num_col])
    df.loc[:, cat_cols] = imputers['cat_imputer'].transform(df.loc[:, cat_cols])

    # Feature engineering Part
    df.loc[:, 'FirstName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[0])
    df.loc[:, 'LastName'] = df.loc[:, 'Name'].apply(lambda x: x.split()[1])

    df.loc[:, 'PassengerGGGG'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[0]))
    df.loc[:, 'PassengerPP'] = df.loc[:, 'PassengerId'].apply(lambda x: int(x.split('_')[1]))
    
    df.loc[:, 'NumberOfFellows'] = df.groupby('PassengerGGGG')['PassengerGGGG'].transform('count')
    df.loc[:, 'IsAlone'] = df.loc[:, 'NumberOfFellows'].apply(lambda x: 1 if x == 1 else 0)

    df['cabin_deck'] = df['Cabin'].apply(lambda x: x.split('/')[0] if x != 'Unknown' else 'U')
    df['cabin_num'] = df['Cabin'].apply(lambda x: int(x.split('/')[1]) if x != 'Unknown' else -1)
    df['cabin_side'] = df['Cabin'].apply(lambda x: x.split('/')[2] if x != 'Unknown' else 'U')

    df.loc[:, 'TotalSpend'] = df.loc[:, ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    df['IsSpendingZero'] = (df['TotalSpend'] == 0).astype(int)

    # Removing unnecessary columns
    df.drop(['Cabin', 'Name', 'PassengerId'], inplace=True, axis=1)
    df.drop(['FirstName', 'LastName', 'PassengerGGGG'], inplace=True, axis=1)

    # Encoding
    cat_data = df.select_dtypes('object')
    cat_data.loc[:, ['CryoSleep', 'VIP']] = cat_data.loc[:, ['CryoSleep', 'VIP']].astype('str')
        
    cat_data_encoded = one_hot_encoder.transform(cat_data)
    encoded_feature_names = one_hot_encoder.get_feature_names_out(cat_data.columns)
    
    cat_encoded = pd.DataFrame(cat_data_encoded.toarray(), columns = encoded_feature_names, index=cat_data.index)
    
    non_cat = df.drop(cat_data.columns, axis=1)
    df = pd.concat([non_cat, cat_encoded], axis=1)
    
    return df

In [6]:
df_train_updated, train_imputers, train_ohe = update_train_df(df_train) 

/tmp/ipykernel_131002/3056761461.py:68: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 1 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, 'Transported'] = df.loc[:, 'Transported'].astype(int)


In [7]:
df_test_updated = update_test_df(df_test, train_imputers, train_ohe)

In [8]:
X_train = df_train_updated.drop('Transported', axis=1)
y_train = df_train_updated.loc[:, 'Transported']

X_test = df_test_updated.drop('Transported', axis=1)
y_test = df_test_updated.loc[:, 'Transported']

In [9]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 0.5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 0.5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 0.5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }
    
    model = XGBClassifier(**params, random_state=42, verbosity=0)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("Best trial:")
print(study.best_trial.params)

[I 2025-07-07 23:19:50,353] A new study created in memory with name: no-name-003cdbae-4ef8-4d87-8aeb-f66507147e72
[I 2025-07-07 23:19:51,136] Trial 0 finished with value: 0.800834035862611 and parameters: {'n_estimators': 322, 'max_depth': 6, 'learning_rate': 0.19263048566039379, 'subsample': 0.7956179418630394, 'colsample_bytree': 0.87274067183877, 'gamma': 0.026700277670561157, 'reg_alpha': 0.14943824383421894, 'reg_lambda': 0.04856864590458354, 'min_child_weight': 14}. Best is trial 0 with value: 0.800834035862611.
[I 2025-07-07 23:19:51,514] Trial 1 finished with value: 0.7896172206734972 and parameters: {'n_estimators': 147, 'max_depth': 7, 'learning_rate': 0.46906416694649994, 'subsample': 0.6902948675415796, 'colsample_bytree': 0.9079829110382963, 'gamma': 0.0715656458103956, 'reg_alpha': 0.32849014788082337, 'reg_lambda': 0.016414616593983633, 'min_child_weight': 13}. Best is trial 0 with value: 0.800834035862611.
[I 2025-07-07 23:19:52,005] Trial 2 finished with value: 0.78803

Best trial:
{'n_estimators': 817, 'max_depth': 3, 'learning_rate': 0.0816525631511912, 'subsample': 0.9618214514489957, 'colsample_bytree': 0.6390303515291257, 'gamma': 0.2256057696902123, 'reg_alpha': 0.40266566984067254, 'reg_lambda': 0.14568814833206822, 'min_child_weight': 3}


In [10]:
study.best_value

0.8101804508944964

In [11]:
train, train_imputers, train_ohe = update_train_df(full_df)

/tmp/ipykernel_131002/3056761461.py:68: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 1 0 ... 1 0 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, 'Transported'] = df.loc[:, 'Transported'].astype(int)


In [12]:
test_df = pd.read_csv('dataset/test.csv')
test = update_test_df(test_df, train_imputers, train_ohe)

In [13]:
X = train.drop('Transported', axis=1)
y = train.loc[:, 'Transported']

In [15]:
best_params = study.best_trial.params

In [16]:
best_model = XGBClassifier(**best_params)
best_model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6390303515291257, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, feature_weights=None,
              gamma=0.2256057696902123, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0816525631511912,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=817, n_jobs=None,
              num_parallel_tree=None, ...)

In [17]:
y_pred = best_model.predict(test)
y_pre = y_pred.astype(bool)

In [18]:
test_df = pd.read_csv('dataset/test.csv')


submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported' : y_pred.astype(bool)
})
submission

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [19]:
submission.to_csv('xgboost-81.01.csv', index=False)

In [20]:
best_params

{'n_estimators': 817,
 'max_depth': 3,
 'learning_rate': 0.0816525631511912,
 'subsample': 0.9618214514489957,
 'colsample_bytree': 0.6390303515291257,
 'gamma': 0.2256057696902123,
 'reg_alpha': 0.40266566984067254,
 'reg_lambda': 0.14568814833206822,
 'min_child_weight': 3}